In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string
from urllib.parse import urlparse
import contractions

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Thapar 4th Sem/Machine Learing/Machine_Learning_Project/data/20221106_215110_tweets.csv')
tweets_df.head()

,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions,tweet_id_dup
0,7.334197e+07,1.589270e+18,ravi4354,"India,Not in Hindu Rashtra",634,6419,2022-11-06 15:05:20+00:00,41202,0,This is the Goodwill earned by Mr @narendramod...,[],"[{'screen_name': 'narendramodi', 'name': 'Nare...",1.589273e+18
1,7.046310e+17,1.589270e+18,iamanshulthakur,"सिवनी,मध्यप्रदेश,भारत",396,2386,2022-11-06 15:04:52+00:00,9400,0,@PrajwalBusta @narendramodi It's very special ...,[],"[{'screen_name': 'PrajwalBusta', 'name': 'Praj...",1.589273e+18
2,1.113370e+18,1.589270e+18,LifestyleVishnu,"Ambikapur, India",304,286,2022-11-06 15:04:46+00:00,5764,0,@AjayHimatlal @YssSpeaks @BJP4Haryana @cmohry ...,"[{'text': 'NoActionOnArrestWarrant', 'indices'...","[{'screen_name': 'AjayHimatlal', 'name': 'Ajay...",1.589273e+18
3,2.249900e+09,1.589270e+18,prkgarg,GLOBE,709,203,2022-11-06 15:04:40+00:00,24799,0,@ArvindKejriwal You will not get more than 2 %...,[],"[{'screen_name': 'ArvindKejriwal', 'name': 'Ar...",1.589273e+18
4,7.808170e+17,1.589270e+18,SunilBhatM,"New Delhi, India",930,2284,2022-11-06 15:04:38+00:00,8235,0,Magnificent!\n\nI visited Pradhan Manthri Sang...,[],"[{'screen_name': 'narendramodi', 'name': 'Nare...",1.589273e+18


In [10]:
tweets_df.shape

(8000, 13)

In [11]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_df.drop_duplicates(subset=['tweet_id_dup'], inplace=True)
print('Shape of dataset after removal of duplicates is {}'.format(tweets_df.shape))

Shape of dataset before removal of duplicates is (8000, 13)
Shape of dataset after removal of duplicates is (2333, 13)


In [12]:
tweets_df.dtypes

user_id           float64
tweet_id          float64
username           object
location           object
following           int64
followers           int64
twt_created_at     object
total_tweets        int64
retweet_count       int64
text               object
hashtags           object
mentions           object
tweet_id_dup      float64
dtype: object

In [13]:
def data_preprocessing(df):
    """
    One function to rule them all, 
    one function to find them, 
    One function to bring them all, 
    and in the darkness bind them; 
    """
    print('Shape of dataset before removal of tweets with less than 5 words is {}'.format(df.shape))
    
    for tweets in df.loc[:,'text']:
        # count+=1
        # print(tweets)
        tokenizer = TweetTokenizer()
        tweet_id = df.loc[df['text'] == tweets, 'tweet_id_dup'].values[0]
        # print(tweet_id)
        
        # removing links
        list_words = tokenizer.tokenize(tweets)
        x = [word for word in list_words if not urlparse(word).scheme]
        tweets = ' '.join(x)

        # contractions handling
        list_words = tokenizer.tokenize(tweets)
        new_list_words = []
        for word in list_words:
            new_list_words.append(contractions.fix(word))
        list_words = new_list_words
        del(new_list_words)
        tweets = ' '.join(list_words)
        
        # adding space between words and punctuations
        tweets = tweets.replace(',', ' ,').replace('.', ' .').replace('?', ' ?').replace('!', ' !')
        
        # removing hashtags and mentions
        list_words = tokenizer.tokenize(tweets)
        list_words = [word for word in list_words if word[0] not in ('#', '@')]
        list_words = [word for word in list_words if word[0] not in ('▪')]
        tweets = ' '.join(list_words)
        
        # removing punctuations
        tweets = tweets.translate(str.maketrans('', '', string.punctuation))
        
        #removing emojis
        tweets = re.sub(r'[^\x00-\x7F]+', ' ', tweets)
        
        #lower case
        tweets = tweets.lower()
        
        #remove numbers
        tweets = re.sub(r'\d+', '', tweets)
        tweets = re.sub(' +', ' ', tweets)
        
        #removing stopwords
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if word not in stopwords.words('english')]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        #lemmatization
        lem = WordNetLemmatizer()
        list_words = tokenizer.tokenize(tweets)
        for word in list_words:
            list_words = list(map(lambda x: x.replace(word, lem.lemmatize(word)), list_words))
        tweets = ' '.join(list_words)
        
        #removing individual letters
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if len(word)>2]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        # updating tweets in dataframe
        df.loc[df['tweet_id_dup']==tweet_id, 'text'] = tweets
        
        #remove small tweets
        list_words = tokenizer.tokenize(tweets)
        if len(list_words) <= 5:
            ind_num = df[df['tweet_id_dup']==tweet_id].index
            df.drop(ind_num, inplace=True)
        # break
    print('Shape of dataset after removal of tweets with less than 5 words is {}'.format(df.shape))
    
    return df
        
        
        

In [14]:
tweets_df = data_preprocessing(tweets_df)

Shape of dataset before removal of tweets with less than 5 words is (2333, 13)
Shape of dataset after removal of tweets with less than 5 words is (1574, 13)


In [15]:
# converting tweets_df into a csv file
filename = '/Users/nitanshjain/Documents/Thapar 4th Sem/Machine Learing/Machine_Learning_Project/data/tweets_preprocessed_1.csv'
tweets_df.to_csv(filename, index=False)